In [1]:
! pip install scikit-image

    100% |████████████████████████████████| 25.4MB 1.6MB/s eta 0:00:01
    100% |████████████████████████████████| 1.6MB 22.6MB/s ta 0:00:01
    100% |████████████████████████████████| 655kB 31.8MB/s ta 0:00:01
    100% |████████████████████████████████| 2.0MB 20.4MB/s ta 0:00:01
    100% |████████████████████████████████| 5.7MB 9.0MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 26.9MB/s ta 0:00:01
  Running setup.py bdist_wheel for networkx ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e
  Running setup.py bdist_wheel for toolz ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
Successfully built networkx toolz
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import os
import glob
import sys
import random
import time
import scipy
from keras.optimizers import  Adam
from random import shuffle
import sys
MY_UTILS_PATH = "../Modules/"
if not MY_UTILS_PATH in sys.path:
    sys.path.append(MY_UTILS_PATH)
import frequency_spatial_network as fsnet  
from scipy.io import loadmat
import undersampling as under
import datetime
import skimage.measure as meas
from keras.callbacks import ModelCheckpoint, EarlyStopping

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
model = fsnet.fs_rec_unet(H=256,W=256,channels = 2,kshape = (3,3))
opt = Adam(lr=0.0001)#0.000025
model.compile(loss = ['mse','mse'],optimizer=opt, loss_weights=[0.01, 0.99])
#model.load_weights("../Data/fs_unet_25perc_under_best.hdf5")
print model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 2)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 48) 912         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 48) 20784       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 256, 256, 48) 20784       conv2d_2[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [4]:
epochs = 100
batch_size=2

# Train Set 
kspace_files = np.asarray(glob.glob("../../Raw-data/Kspace/Train/*.npy"))
rec_files = np.asarray(glob.glob("../../Raw-data/reconstructed/Train/*.npy"))
indexes = np.arange(rec_files.size,dtype = int)
np.random.shuffle(indexes)
kspace_files_train = kspace_files[indexes]
rec_files_train = rec_files[indexes]

# Validation set
kspace_files_val = np.asarray(glob.glob("../../Raw-data/Kspace/Val/*.npy"))
rec_files_val = np.asarray(glob.glob("../../Raw-data/reconstructed/Val/*.npy"))

print kspace_files_train[0]
print rec_files_train[0]
print len(rec_files_train)
print len(kspace_files_train)
print 
print kspace_files_val[0]
print rec_files_val[0]

../../Raw-data/Kspace/Train/e14079s3_P09216.7.npy
../../Raw-data/reconstructed/Train/e14079s3_P09216.7.npy
17
17

../../Raw-data/Kspace/Val/e14258s3_P76800.7.npy
../../Raw-data/reconstructed/Val/e14258s3_P76800.7.npy


In [ ]:
imshape = (256,256)
norm = np.sqrt(imshape[0]*imshape[1])
nchannels = 2 #complex data real + imag

var_sampling_mask = (under.var_dens_mask(imshape,0.000165,True))
var_sampling_mask = np.fft.fftshift(~var_sampling_mask) == -1
            
np.save("../Data/sampling_mask_25perc.npy",var_sampling_mask)

print "Undersampling:", 1.0*var_sampling_mask.sum()/var_sampling_mask.size
print "Mask type:",  var_sampling_mask.dtype
plt.figure()
plt.imshow(~var_sampling_mask,cmap = "gray")
plt.axis("off")
plt.show()

In [ ]:
# Training
ntrain = 0
for ii in xrange(len(kspace_files_train)):
    ntrain += np.load(kspace_files_train[ii]).shape[0]

kspace_train = np.zeros((ntrain,imshape[0],imshape[1],nchannels))
rec_train = np.zeros((ntrain,imshape[0],imshape[1],1))
aux_counter = 0
for ii in xrange(len(kspace_files_train)):
     aux_kspace = np.load(kspace_files_train[ii])/norm
     aux = aux_kspace.shape[0]   
     kspace_train[aux_counter:aux_counter+aux] = aux_kspace
     rec_train[aux_counter:aux_counter+aux,:,:,0] = \
     np.abs(np.fft.ifft2(aux_kspace[:,:,:,0]+1j*aux_kspace[:,:,:,1]))
     aux_counter+=aux
# Shuffle training    
indexes = np.arange(rec_train.shape[0],dtype = int)
np.random.shuffle(indexes)
rec_train = rec_train[indexes]
kspace_train = kspace_train[indexes]

# Undersampling kspace
kspace_train2 = kspace_train.copy()
kspace_train[:,var_sampling_mask,:] = 0

print "Number of training samples", kspace_train.shape[0]
print "Kspace full stats", kspace_train.mean(),kspace_train.std()
print "Kspace under stats", kspace_train2.mean(),kspace_train2.std()
print "Rec stats", rec_train.mean(),rec_train.std()

In [ ]:
plt.figure()
plt.subplot(131)
plt.imshow(np.log(1+ np.abs(kspace_train[1500,:,:,0] + \
                         1j*kspace_train[1500,:,:,1])),cmap = "gray")
plt.subplot(132)
plt.imshow(np.abs(np.fft.ifft2(kspace_train[1500,:,:,0] + \
                         1j*kspace_train[1500,:,:,1])),cmap = "gray")
plt.subplot(133)
plt.imshow(rec_train[1500,:,:,0],cmap = "gray")
plt.show()

In [ ]:
# Validation
nval = 0
for ii in xrange(len(kspace_files_val)):
    nval += np.load(kspace_files_val[ii]).shape[0]

kspace_val = np.zeros((nval,imshape[0],imshape[1],nchannels))
rec_val = np.zeros((nval,imshape[0],imshape[1],1))
aux_counter = 0
for ii in xrange(len(kspace_files_val)):
    aux_kspace = np.load(kspace_files_val[ii])/norm
    aux = aux_kspace.shape[0]   
    kspace_val[aux_counter:aux_counter+aux] = aux_kspace
    rec_val[aux_counter:aux_counter+aux,:,:,0] = \
    np.abs(np.fft.ifft2(aux_kspace[:,:,:,0]+1j*aux_kspace[:,:,:,1]))
    aux_counter+=aux

# Undersampling kspace
kspace_val2 = kspace_val.copy()
kspace_val[:,var_sampling_mask,:] = 0

print "Number of samples", kspace_val.shape[0]
print "Kspace under stats", kspace_val.mean(),kspace_val.std()
print "Kspace full stats", kspace_val2.mean(),kspace_val2.std()
print "Rec stats", rec_val.mean(),rec_val.std()

In [ ]:
plt.figure()
plt.subplot(131)
plt.imshow(np.log(1+ np.abs(kspace_val[1100,:,:,0] + \
                         1j*kspace_val[1100,:,:,1])),cmap = "gray")
plt.subplot(132)
plt.imshow(np.abs(np.fft.ifft2(kspace_val[1100,:,:,0] + \
                         1j*kspace_val[1100,:,:,1])),cmap = "gray")
plt.subplot(133)
plt.imshow(rec_val[1100,:,:,0],cmap = "gray")
plt.show()

In [ ]:
model_name = "../Data/fs_unet_25perc_under_best.hdf5"

# Early stopping callback to shut down training after
#10 epochs with no improvement
earlyStopping = EarlyStopping(monitor='val_loss',
                                       patience=5, 
                                       verbose=0, mode='min')

# Checkpoint callback to save model  along the epochs
checkpoint = ModelCheckpoint(model_name, mode = 'min', \
                             monitor='val_loss',verbose=0,\
                             save_best_only=True, save_weights_only = True)

In [ ]:
model.fit(kspace_train,[kspace_train2,rec_train],epochs=100,\
          batch_size = 8,verbose = 1, \
          validation_data = (kspace_val,[kspace_val2,rec_val]),\
          callbacks=[checkpoint,earlyStopping])